### Extract from OSM
The purpose of this script is to show how to go from a .osm.pbf file to a network format that GOSTnets will accept. 
Additional modifications in this script include: clipping the roads to an input polygon (here, kinshasa.shp), and slicing the roads DataFrame by a list of accepted road types, as tagged in OSM (see accepted_road_types list). 

In [1]:
import geopandas as gpd
import pandas as pd
import os, sys
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn
import LoadOSM as losm
import importlib

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


Define filepaths

In [2]:
pth = r'C:\Users\charl\Documents\GOST\Kinshasa'
fil = r'congo-democratic-republic-latest.osm.pbf'
f = os.path.join(pth, fil)

Using the loadOSM library, pass the filepath of the .osm.pbf to initiate a OSM_to_network class

In [5]:
kin = losm.OSM_to_network(f)

This generates a property of the class called 'roads_raw'. This is a raw dataframe of the highway objects, extracted from the .osm.pbf. This is the starting point for our network

In [6]:
kin.roads_raw.infra_type.value_counts()

residential       159759
path               68975
unclassified       66156
track              54540
service             8412
tertiary            8353
road                5011
secondary           4212
primary             3992
footway             1749
trunk                914
living_street        178
proposed              91
primary_link          81
trunk_link            69
tertiary_link         67
secondary_link        63
bridleway             62
pedestrian            31
steps                 12
yes                    4
construction           4
raceway                1
cycleway               1
Name: infra_type, dtype: int64

In [7]:
accepted_road_types = ['residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link']

We call the filerRoads method and pass it a list of acceptable road types

In [8]:
kin.filterRoads(acceptedRoads = accepted_road_types)

In [9]:
kin.roads_raw.infra_type.value_counts()

residential       159759
unclassified       66156
track              54540
service             8412
tertiary            8353
road                5011
secondary           4212
primary             3992
trunk                914
primary_link          81
trunk_link            69
tertiary_link         67
secondary_link        63
Name: infra_type, dtype: int64

Here, we import a shapefile for our AOI (Kinshasa in the DRC) and clip to its extent using standard GeoPandas functions

In [10]:
shp = gpd.read_file(os.path.join(pth, 'Kinshasa.shp'))
shp = shp.to_crs({'init':'epsg:4326'})
shp_obj = shp.geometry.iloc[0]

In [11]:
kin.roads_raw.geometry.intersects(shp_obj).value_counts()

False    279233
True      32396
dtype: int64

In [12]:
kin.roads_raw = kin.roads_raw.loc[kin.roads_raw.geometry.intersects(shp_obj) == True]

having narrowed the DataFrame to the roads we are interested in, we generate the RoadsGDF object

In [13]:
kin.generateRoadsGDF(verbose = False)

With initialReadIn(), we transform this to a graph object

In [15]:
kin.initialReadIn()

We save this graph object down to file using gn.save()

In [16]:
gn.save(kin.network,'kinshasa',pth)